# PowerShell User Group innsalzach 08.07.2021

## PowerShell DSC with Azure Automation DSC

## What is DSC?

DSC (Desired State Configuration) is a platform where you can manage and develop your infrastructure with Configuration as Code. You write a DSC Configuration in a declarative way to describe the target state of your Infrastructure.

### Declarative vs. Imperative Code

I used declarative above but have not told you about what that means.

**Imperative:**

PowerShell executes what we develop and executes It. We as developers have to for example care about the error handling and the repetetive execution. An example of imperative code would be:

In [ ]:
#Check and install ASP.NET 4.5 feature
If (-not (Get-WindowsFeature "Web-Asp-Net45").Installed) {
    try {
        Add-WindowsFeature Web-Asp-Net45
    }
    catch {
        Write-Error $_
    }
}

**Declarative:**

We as developers define what needs to be done and not how it needs to be done. We do not care how to get to the desired state. The underlying platform (DSC) has to know that. An example of declarative code would be:

In [ ]:
Configuration WebSiteConfig
{
    Node MyWebServer
    {
        WindowsFeature IIS
        {
            Ensure = "Present"
            Name = "Web-Asp-Net45"
        }      
    }
}

### Important Terms in DSC

**Configurations:**

The declarative PowerShell scripts which define the desired state for the infrastructure. The configurations calls the DSC resources to bring the system in the desired state ("make it so").

They get "compiled" into a MOF (Managed Object Format) - File. The MOF - File is executed by the LCM.

**Resource:**

They contain the code that put and keep the target of a configuration in the specified state.

Resouces are PowerShell modules with a specific file structure.

Every resource has the following functions / methods:

- Get
- Set
- Test

**LCM (Local Configuration Manager):**

This is the core of PowerShell DSC. It is installed on every Windows System that has PowerShell 4.0 and above installed. It executes the MOF - Files and brings the infrastructure in its desired state. If the infrastructure is not in the desired state the LCM will trigger the last know configuration and put the system back in the desired state (Configuration drift).

  

The state of the LCM can be checked with

In [ ]:
Get-DscLocalConfigurationManager

If you want to configure the LCM you have to write a special mof file (meta.mof).

After that you execute

In [ ]:
Set-DscLocalConfigurationManager -Path <Path to meta.mof (not with file name)>

If you want to read further [Configure LCM](https://nedimmehic.org/2019/05/06/desired-state-configuration-dsc-local-configuration-manager-lcm/).

**Configuration Data**

In DSC you normally seperate the configuration and the configuration data. The configuration data is a .ps1 - File and the configuration data is a .psd1 file (PowerShell Data File)

  

What that means is that you can reuse your DSC Configuration across multiple environments (DEV, TEST, PROD) and only replace the configuration Data (configuration Information). A sample configuration Data file:

In [ ]:
$MyData =
@{
    AllNodes =
    @(
        @{
            NodeName           = "*"
            LogPath            = "C:\Logs"
        },

        @{
            NodeName = "VM-1"
            SiteContents = "C:\Site1"
            SiteName = "Website1"
        },


        @{
            NodeName = "VM-2"
            SiteContents = "C:\Site2"
            SiteName = "Website2"
        }
    );

    NonNodeData =
    @{
        ConfigFileContents = (Get-Content C:\Template\Config.xml)
     }
}

The configuration File referencing the configuration data

In [ ]:
configuration WebsiteConfig
{
    Import-DscResource -ModuleName xWebAdministration

    node $AllNodes.NodeName
    {
        xWebsite Site
        {
            Name         = $Node.SiteName
            PhysicalPath = $Node.SiteContents
            Ensure       = "Present"
        }

        File ConfigFile
        {
            DestinationPath = $Node.SiteContents + "\\config.xml"
            Contents = $ConfigurationData.NonNodeData.ConfigFileContents
        }
    }
}

Configuration data can get complex very fast. If you want to manage your configuration data right please use Gaels module [Datum.](https://github.com/gaelcolas/Datum)

  

Also please have a look at the DSC Workshop repository ([DSCWorkshop](https://github.com/dsccommunity/DscWorkshop/)). This project uses Datum to build the MOF-Files.

**Push and Pull Configuration**

**Push Configuration**

This is the easiest method of executing a DSC configuration against one or more nodes. It is the best solution to test your DSC configuration because you have to run the configuration manually. For initiating the configuration you have to run

In [ ]:
Start-DSCConfiguration [-ComputerName] -Path (Path to the folder where the DSC Configs are in)

**Pull Configuration**

If you want to use pull mode you need a central Windows Server (or Azure Automation DSC) with the Windows DSC feature installed. You call this server the 'Pull Server'. One advantage over the push mode is if a node is not reachable the config will run after the node is reachable again, because the node checks every 15 minutes if there is a change in the nodes configuration and apply it. If you use the push mode you have to check when the node is reachable again and run the DSC configuration manually.

All DSC Configurations and the dependent modules are stored on that 'Pull Server'. So you have a central place for all your node configurations.

You can setup a pull server as in two ways

- SMB
- http/https

The HTTP Server is mostly used in environments. It is a an IIS Webserver with 2 Websites installed. You can (since Server 2016) install it via Add- Remove Windows Features or via DSC.

  

The 2 Websites are:

- PSDSCPullServer: Will allow nodes to download their configurations
- PSDSCComplianceServer: Nodes sent the status to this endpoint so the Compliance Server can react if there is a configuration drift.

We will not install a DSC Pull Server today, because we are using Azure Automation DSC today.

  

If you want to know how to host your own 'Pull Server' you can use this manual ([Install DSC Pull Server](https://www.red-gate.com/simple-talk/sysadmin/powershell/powershell-desired-state-configuration-pull-mode/))

## Azure Automation DSC

Azure Automation DSC is a configuration management service that allows you to write, manage, and compile PowerShell Desired State Configuration (DSC)  for nodes in any cloud or on-premises datacenter. It is included in an Automation Account.

### Why Azure Automation State Configuration?

You do not need to host your own service (scalability and secure). The biggest advantage is the rich reporting of Azure Automation DSC because the Windows Server Pull Server does not have this reporting. You have to built the reporting on your own. It also has the Pull Server already set up.

# DEMO TIME